<a href="https://colab.research.google.com/github/sowji2604/MyPhp/blob/main/IMAGE_RECOGNITION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
!pip install tensorflow


In [131]:
!pip install numpy

In [132]:
!pip install streamlit

In [133]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img , ImageDataGenerator
from tensorflow.keras.models import Sequential , load_model
from tensorflow.keras.layers import Dense , Dropout , Flatten , Conv2D , MaxPooling2D

In [134]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abhinavkrjha/dog-breed-classification")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/dog-breed-classification


In [135]:
dirs = os.listdir('/kaggle/input/dog-breed-classification')
for dir in dirs:
  files = list(os.listdir('/kaggle/input/dog-breed-classification/'+dir))
  print(dir + ' Folder has: ' + str(len(files))+'images')

Afgan Hound dog Folder has: 50images
Bulldog dog Folder has: 50images
Beagle dog Folder has: 50images
Borzoi dog Folder has: 50images
Bichon Frise dog Folder has: 50images
American Bulldog dog Folder has: 48images
Alaskan Malamute dog Folder has: 50images
Bocker dog Folder has: 50images
Affenhuahua dog Folder has: 50images
Bugg dog Folder has: 50images
Belgian Tervuren dog Folder has: 50images
Boxer dog Folder has: 50images
Auggie dog Folder has: 50images
Akita dog Folder has: 49images


In [136]:
base_dir = '/kaggle/input/dog-breed-classification'
img_size = 180
batch = 32

In [137]:
train_ds =tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(img_size,img_size),
    batch_size=batch
)

val_ds =tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(img_size,img_size),
    batch_size=batch
)

Found 697 files belonging to 14 classes.
Using 558 files for training.
Found 697 files belonging to 14 classes.
Using 139 files for validation.


In [138]:
dog_names = train_ds.class_names
dog_names

['Affenhuahua dog',
 'Afgan Hound dog',
 'Akita dog',
 'Alaskan Malamute dog',
 'American Bulldog dog',
 'Auggie dog',
 'Beagle dog',
 'Belgian Tervuren dog',
 'Bichon Frise dog',
 'Bocker dog',
 'Borzoi dog',
 'Boxer dog',
 'Bugg dog',
 'Bulldog dog']

In [139]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [140]:
data_augmentation = Sequential([
    layers.RandomFlip('horizontal',input_shape=(img_size,img_size,3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

In [141]:
model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10)
])

In [142]:
# ... (previous code) ...

# Assuming there are 120 breeds in the dataset
num_classes = 120

model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes) # Changed to num_classes
])

# ... (rest of the code) ...

In [143]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [144]:
model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_13 (Sequential)      │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_9 (Rescaling)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 180, 180, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 90, 90, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 90, 90, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 45, 45, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 45, 45, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │     3,965,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 120)            │        15,480 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,004,120 (15.27 MB)

 Trainable params: 4,004,120 (15.27 MB)

 Non-trainable params: 0 (0.00 B)

In [145]:
history = model.fit(train_ds,validation_data=val_ds,epochs=15)

Epoch 1/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.0398 - loss: 3.8774 - val_accuracy: 0.0863 - val_loss: 2.8608
Epoch 2/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.0980 - loss: 2.6848 - val_accuracy: 0.0647 - val_loss: 2.6990
Epoch 3/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.1579 - loss: 2.6064 - val_accuracy: 0.1079 - val_loss: 2.7038
Epoch 4/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.1569 - loss: 2.4837 - val_accuracy: 0.1871 - val_loss: 2.6554
Epoch 5/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.2581 - loss: 2.3109 - val_accuracy: 0.1511 - val_loss: 2.5992
Epoch 6/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.2827 - loss: 2.1652 - val_accuracy: 0.2662 - val_loss: 2.6611
Epoch 7/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.3727 - loss: 2.0475 - val_accuracy: 0.2302 - val_loss: 2.5163
Epoch 8/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.3713 - loss: 1.8748 - val_accuracy: 0.2590 - val_loss:

In [146]:
def classify_image(image_path):
  input_image = tf.keras.utils.load_img(image_path, target_size=(180,180))
  input_image_array = tf.keras.utils.img_to_array(input_image)
  input_image_exp_dim = tf.expand_dims(input_image_array,0)
  predictions = model.predict(input_image_exp_dim)
  result = tf.nn.softmax(predictions[0])
  outcome= 'the dog belong to '+ dog_names[np.argmax(result)] + ' with a score of '+str(max(result)*100)
  return outcome

In [147]:
classify_image('/kaggle/input/dog-breed-classification/Afgan Hound dog/Image_1.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


'the dog belong to Afgan Hound dog with a score of tf.Tensor(37.685204, shape=(), dtype=float32)'

In [148]:
import os

afghan_hound_dir = '/kaggle/input/dog-breed-classification/Afgan Hound dog'
if os.path.exists(afghan_hound_dir):
    print(os.listdir(afghan_hound_dir))
else:
    print(f"Directory not found: {afghan_hound_dir}")

['Image_22.jpg', 'Image_4.jpg', 'Image_12.jpg', 'Image_16.jpg', 'Image_2.jpg', 'Image_45.jpg', 'Image_7.jpg', 'Image_5.jpg', 'Image_36.jpg', 'Image_18.jpg', 'Image_33.jpg', 'Image_19.jpg', 'Image_31.jpg', 'Image_27.jpg', 'Image_11.jpg', 'Image_39.jpg', 'Image_26.jpg', 'Image_28.jpg', 'Image_44.jpg', 'Image_14.jpg', 'Image_48.jpg', 'Image_41.jpg', 'Image_10.jpg', 'Image_47.jpg', 'Image_30.jpg', 'Image_23.jpg', 'Image_34.jpg', 'Image_50.jpg', 'Image_32.jpg', 'Image_21.jpg', 'Image_25.jpg', 'Image_17.jpg', 'Image_15.jpg', 'Image_1.jpg', 'Image_13.jpg', 'Image_43.jpg', 'Image_40.jpg', 'Image_35.jpg', 'Image_42.jpg', 'Image_8.jpg', 'Image_38.jpg', 'Image_46.jpg', 'Image_49.jpg', 'Image_3.jpg', 'Image_20.jpg', 'Image_24.jpg', 'Image_29.jpg', 'Image_37.jpg', 'Image_6.jpg', 'Image_9.jpg']
